In [0]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/DS8013 Deep Learning/Project')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
np.random.seed(42)

In [0]:
# load and view data
df_blogs = pd.read_csv('blogtext.csv')

df_blogs.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [0]:
# take post contents and user id: the X and y for this task
df_Xy = df_blogs[['text', 'id']]

df_Xy.head()

,text,id
0,"Info has been found (+/- 100 pages,...",2059027
1,These are the team members: Drewe...,2059027
2,In het kader van kernfusie op aarde...,2059027
3,testing!!! testing!!!,2059027
4,Thanks to Yahoo!'s Toolbar I can ...,3581210


In [0]:
# find the 10 users with the most blog posts
top_10 = df_Xy['id'].value_counts()[:10]

top_10

449628     4221
734562     2301
589736     2294
1975546    2261
958176     2244
1107146    2237
303162     2114
942828     2068
1270648    1951
1784456    1843
Name: id, dtype: int64

In [0]:
# total number of posts
top_10.sum()

23534

In [0]:
# save the user id of the top 10 users
top_id = list(top_10.index)

print(top_id)

[449628, 734562, 589736, 1975546, 958176, 1107146, 303162, 942828, 1270648, 1784456]


In [0]:
# get posts from the top 10 users
df_top = df_Xy.loc[df_Xy['id'].isin(top_id)]

df_top.head()

,text,id
1393,Much funny. 2 points. As mentioned in the...,589736
1394,"Harpers, Harpers, everywhere. Harpers, Har...",589736
1395,"In an earlier post, Johnathan said: 'And ...",589736
1396,"I'd post this on the RTG Blog, but I can't...",589736
1397,The answer to the first question lies with ...,589736


In [0]:
texts = df_top['text'].values

print(texts[:2])

["   Much funny.  2 points.  As mentioned in the email, no game this week; I have to research a 13th century poem for a final paper.  I enjoy the work, but I'd much rather have swashbuckling adventure in wintery Hillsfar. "
 "   Harpers, Harpers, everywhere.  Harpers, Harpers, they really care. Harpers, Harpers, stay in motion.  Harpers, Harpers, healing potions. Harpers, Harpers, pins of green.  Harpers, Harpers, enter the scene. Harpers, Harpers, formed by a bard.  Harpers, Harpers, .... What?  A Bard?!?!  You got to be fucking kidding me, right?!?!  A Bard?!  Jesus Christ, we're screwed!! "]


In [0]:
ids = df_top['id'].values

ids

array([ 589736,  589736,  589736, ..., 1270648, 1270648, 1270648])

In [0]:
# split test, train, validation
X, X_test, y, y_test = train_test_split(texts, ids, test_size=0.1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [0]:
# custom tokenizer. gets alphanumeric strings and stems resulting words
def myTokenizer(textData):
    tokenizer = RegexpTokenizer('[^_\W]+')
    stemmer = PorterStemmer()
    stemmedWords = [stemmer.stem(w) for w in tokenizer.tokenize(textData)]
    return  stemmedWords

# stopwords
stops = set(stopwords.words('english'))
print ("Stopwords: ", stops)

# vocab size 10000
vect = CountVectorizer(tokenizer=myTokenizer, stop_words=stops, 
                       max_features=10000)
vect.fit(X_train)
print ("Vocabulary size: ", len(vect.vocabulary_))

Stopwords:  {'theirs', 'which', 'all', 'with', "didn't", 'and', 'mightn', 'further', 'on', 'itself', 'having', 'then', 'o', 'shouldn', 'to', 'wasn', "hasn't", 'above', 'very', 'been', 'against', "haven't", 'herself', 'some', 'own', 'couldn', 'most', 'your', 'myself', 'themselves', 'a', 'had', 'do', 'shan', 'nor', 'won', 'up', 'during', "don't", "wouldn't", "should've", "couldn't", "she's", 'isn', 'of', "it's", 'as', "shouldn't", 'i', 'y', 'ain', 'will', 'about', 'before', 'now', 'have', 'those', 'them', 'these', 'such', 'ma', 's', 'yourselves', 'wouldn', "that'll", 'each', "you'll", 'few', 't', 'be', 'or', 'below', 'doesn', 'over', 'should', "needn't", 'is', 'its', 'both', 'she', "you've", 'we', "mightn't", 'our', 'll', 'hasn', 'from', 'any', 'other', 'whom', 'was', 'were', 'not', 'into', 'd', 'doing', 're', 'here', 'hadn', 'but', 'for', "aren't", "you'd", 'while', 'an', "you're", 'by', "isn't", 'weren', 'the', 'after', 'more', 'he', 'there', 'where', 'haven', 'm', 'until', 'hers', 'th

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Vocabulary size:  10000


In [0]:
# view vocabulary
print(vect.vocabulary_)

{'congrad': 2059, 'faisal': 3352, 'realli': 7248, 'nice': 6098, 'got': 3966, 'good': 3949, 'grade': 3984, 'way': 9652, 'guy': 4097, 'know': 5077, 'care': 1559, 'cours': 2209, 'first': 3506, 'helicopt': 4262, 'like': 5279, 'devic': 2587, 'wa': 9580, 'chines': 1756, 'fourth': 3651, 'centuri': 1652, 'book': 1266, 'call': 1504, 'mean': 5605, 'tell': 8826, 'master': 5565, 'describ': 2551, 'fli': 3542, 'car': 1552, 'cool': 2143, 'heh': 4252, 'kid': 5037, 'believ': 1054, 'non': 6141, 'militari': 5730, 'respons': 7451, 'best': 1089, 'strategi': 8519, 'perman': 6629, 'end': 3086, 'terror': 8863, 'urllink': 9381, '9': 248, 'org': 6341, 'osama': 6357, 'saddam': 7662, 'must': 5960, 'pee': 6582, 'pant': 6476, 'one': 6288, 'rosi': 7591, 'dun': 2885, 'get': 3857, 'name': 5992, 'bold': 1244, 'shock': 8000, 'noth': 6170, 'surpris': 8675, 'anymor': 642, 'humphrey': 4464, 'god': 3925, 'go': 3922, 'watch': 9646, 'walk': 9606, '1000': 21, 'italian': 4824, 'suit': 8620, 'slick': 8149, 'back': 903, 'short': 

In [0]:
# get vector representations
X_train_count = vect.transform(X_train)

In [0]:
# fit naive bayes classifier on training data
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
# get vector representations
X_val_count = vect.transform(X_val)

In [0]:
# score == micro-F1 as below
clf.score(X_val_tfidf, y_val)

0.4197355996222852

In [0]:
pred = clf.predict(X_val_tfidf)

precision, recall, fMeasure, support = \
    precision_recall_fscore_support(y_val, pred, average='micro')

fMeasure

0.4197355996222852

In [0]:
# get vector representations
X_test_count = vect.transform(X_test)

In [0]:
# score == micro-F1 as below
clf.score(X_test_tfidf, y_test)

0.4405267629566695

In [0]:
pred = clf.predict(X_test_tfidf)

precision, recall, fMeasure, support = \
    precision_recall_fscore_support(y_test, pred, average='micro')

fMeasure

0.4405267629566695